## Phase 0: Le Sandbox Poétique



### Chargement du modèle


In [1]:
import sys
from pathlib import Path
from llama_cpp import Llama

# Chemin vers le modèle téléchargé
MODEL_PATH = "./models/Llama-3.2-1B-Instruct.Q4_K_M.gguf"

# Vérifier que le modèle existe
if not Path(MODEL_PATH).exists():
    print(f"erreur: Le modèle n'a pas été trouvé à {MODEL_PATH}")
    print("run python download_model.py")
    sys.exit(1)


# n_ctx=2048 signifie que le modèle peut se souvenir d'environ 1500 mots de conversation
llm = Llama(
    model_path=MODEL_PATH,
    n_ctx=2048,
    verbose=False  # True : voir les statistiques de vitesse
)

print("done")


llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


done


### Sandbox

In [9]:
TOPIC = "le silence de l'hiver"
FORM = "sonnet"  # "sonnet", "limerick", "poème en vers libres"

prompt_structure = [
    {"role": "system", "content": "You are a skilled poet."},
    {"role": "user", "content": f"Write a {FORM} about {TOPIC} in french."}
]

print(f"\n--- Génération d'un {FORM} sur '{TOPIC}' ---\n")

output = llm.create_chat_completion(
    messages=prompt_structure,
    max_tokens=1000,   
    temperature=0.8,
    top_p=0.95,
)

poem = output["choices"][0]["message"]["content"]

print(poem)



--- Génération d'un sonnet sur 'le silence de l'hiver' ---

In le silence de l'hiver, les branches sont froisses,
Les feuilles se sont éteintes, comme des larmes,
Les arbres, qui ont chanté tout l'an,
Se courent, comme des souvenirs qui s'entretiennent.

Le vent souffle, un murmure doux et larme,
Les glaçons qui tombent de l'atmosphère,
Un mélange de glace et de mouchoir de larme,
Qui fait trembler les arbres, comme des éclairs de l'air.

Mais dans cette fin de hiver, je trouve un charme,
Un silence profond, qui me fait se lancer,
Vers les souvenirs, vers les souvenirs de la garde,
Vers les bois qui font chouer, les bois qui font chanter.

Et puis, dans ce silence, je trouve une paix,
Une paix qui me fait oublier mes souvenirs, ma grâce.

(Note: I've tried to follow the traditional sonnet structure and rhyme scheme, but I want to acknowledge that "le silence de l'hiver" is a bit of a challenging topic for a sonnet, as it's more of a poetic description than a traditional narrative. If 

## Sauvegarder le poème 


In [3]:
from pathlib import Path
from datetime import datetime

output_file = Path("poem.txt")
output_file.write_text(poem, encoding="utf-8")



544

### Implémentation du graphe thématique et de la fonction graph_to_prompt


In [6]:
def graph_to_prompt(graph, form_spec):
    """
    Transforme un graphe thématique en prompt structuré pour la génération poétique.
    
    Args:
        graph: dict avec "nodes" (list) et "edges" (list de tuples)
        form_spec: dict avec "form" (str) et "total_lines" (int)
    
    Returns:
        str: prompt structuré détaillant le plan de chaque section
    """
    nodes = graph["nodes"]
    edges = graph["edges"]
    form = form_spec["form"]
    total_lines = form_spec["total_lines"]
    
    # Calculer la distribution des lignes
    num_sections = len(nodes) + len(edges) + 1  # sections pour chaque noeud, chaque transition, + synthèse
    lines_per_section = total_lines // num_sections
    remainder = total_lines % num_sections
    
    prompt_parts = [
        f"You will write a {total_lines}-line {form} in French.",
        "Follow this structured plan for each section:",
        ""
    ]
    
    line_num = 1
    
    # Section pour chaque noeud initial
    for i, node in enumerate(nodes):
        end_line = line_num + lines_per_section - 1
        if i < remainder:
            end_line += 1
        prompt_parts.append(f"Lines {line_num}-{end_line}: Focus on '{node}' imagery and emotions.")
        line_num = end_line + 1
    
    # Sections pour les transitions (edges)
    for i, (source, target) in enumerate(edges):
        end_line = line_num + lines_per_section - 1
        if len(nodes) + i < remainder:
            end_line += 1
        prompt_parts.append(f"Lines {line_num}-{end_line}: Transition from '{source}' to '{target}'.")
        line_num = end_line + 1
    
    # Section de synthèse finale
    if line_num <= total_lines:
        remaining_lines = total_lines - line_num + 1
        all_nodes = ", ".join(nodes)
        prompt_parts.append(f"Lines {line_num}-{total_lines}: Synthesize all themes ({all_nodes}) in a concluding reflection.")
    
    return "\n".join(prompt_parts)

# Exemple de graphe thématique
example_graph = {
    "nodes": ["deuil", "mémoire", "océan"],
    "edges": [("deuil", "mémoire"), ("mémoire", "océan")]
}

# Spécification pour un sonnet (14 lignes)
sonnet_spec = {
    "form": "sonnet",
    "total_lines": 14
}

# Générer le prompt structuré
structured_prompt = graph_to_prompt(example_graph, sonnet_spec)
print("=== Prompt structuré généré ===\n")
print(structured_prompt)


=== Prompt structuré généré ===

You will write a 14-line sonnet in French.
Follow this structured plan for each section:

Lines 1-3: Focus on 'deuil' imagery and emotions.
Lines 4-6: Focus on 'mémoire' imagery and emotions.
Lines 7-8: Focus on 'océan' imagery and emotions.
Lines 9-10: Transition from 'deuil' to 'mémoire'.
Lines 11-12: Transition from 'mémoire' to 'océan'.
Lines 13-14: Synthesize all themes (deuil, mémoire, océan) in a concluding reflection.


### Génération avec le graphe thématique


In [ ]:

TOPIC_GRAPH = {
    "nodes": ["deuil", "mémoire", "océan"],
    "edges": [("deuil", "mémoire"), ("mémoire", "océan")]
}

FORM_SPEC = {
    "form": "sonnet",
    "total_lines": 14
}


structured_instructions = graph_to_prompt(TOPIC_GRAPH, FORM_SPEC)

prompt_with_graph = [
    {"role": "system", "content": "You are a skilled poet. Write in French."},
    {"role": "user", "content": structured_instructions}
]

print("=== Génération avec graphe thématique ===\n")
print("Instructions structurées:")
print(structured_instructions)
print("\n" + "="*50 + "\n")

output_graph = llm.create_chat_completion(
    messages=prompt_with_graph,
    max_tokens=1000,
    temperature=0.8,
    top_p=0.95,
)

poem_with_graph = output_graph["choices"][0]["message"]["content"]
print(poem_with_graph)


=== Génération avec graphe thématique ===

Instructions structurées:
You will write a 14-line sonnet in French.
Follow this structured plan for each section:

Lines 1-3: Focus on 'deuil' imagery and emotions.
Lines 4-6: Focus on 'mémoire' imagery and emotions.
Lines 7-8: Focus on 'océan' imagery and emotions.
Lines 9-10: Transition from 'deuil' to 'mémoire'.
Lines 11-12: Transition from 'mémoire' to 'océan'.
Lines 13-14: Synthesize all themes (deuil, mémoire, océan) in a concluding reflection.


Dans l'ombre sombre où deuil éclate
Les souvenirs se répandent comme des feuilles
Froides et éphémères qui brûlent dans la mémoire
Comme des rêves qui disparaissent dans l'océan

Les ténèbres me recueillent comme des oiseaux
Qui retournent vers leurs ailes mortes, sans voix
Les étoiles éclatent dans l'espace sombre comme des éclairs
Comme des larmes qui ne peuvent plus pleurer dans l'océan

Le temps me frappe comme un tonnerre
Qui me fait trembler sous le poids de mes souvenirs
Les vagues de la